In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28080
2,Huelva,Confirmados PDIA 14 días,1257
3,Huelva,Tasa PDIA 14 días,"244,9480678917318"
4,Huelva,Confirmados PDIA 7 días,559
5,Huelva,Total Confirmados,28253
6,Huelva,Curados,23588
7,Huelva,Fallecidos,363


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28080.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7281.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1219 personas en los últimos 7 días 

Un positivo PCR cada 505 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28080.0,559.0,1257.0,513170.0,108.930764,244.948068,100.0
Huelva-Costa,16437.0,195.0,482.0,289548.0,67.346347,166.466354,67.0
Condado-Campiña,8737.0,292.0,643.0,156231.0,186.902727,411.570047,42.0
Huelva (capital),7281.0,118.0,285.0,143837.0,82.037306,198.140951,39.0
Cartaya,1251.0,31.0,108.0,20083.0,154.359408,537.768262,11.0
Bollullos Par del Condado,781.0,35.0,62.0,14387.0,243.275179,430.944603,8.0
Gibraleón,581.0,17.0,25.0,12737.0,133.469420,196.278559,7.0
Almonte,1445.0,94.0,219.0,24507.0,383.563880,893.622230,7.0
Sierra de Huelva-Andévalo Central,2509.0,65.0,114.0,67391.0,96.452048,169.162054,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres de San Bartolomé,10.0,5.0,5.0,376.0,1329.787234,1329.787234,0.0
Cumbres Mayores,48.0,18.0,23.0,1749.0,1029.159520,1315.037164,0.0
Almonte,1445.0,94.0,219.0,24507.0,383.563880,893.622230,7.0
Santa Ana la Real,9.0,3.0,4.0,475.0,631.578947,842.105263,0.0
Galaroza,27.0,10.0,11.0,1382.0,723.589001,795.947902,3.0
Rociana del Condado,495.0,21.0,61.0,7939.0,264.516942,768.358735,0.0
Lucena del Puerto,207.0,17.0,24.0,3261.0,521.312481,735.970561,5.0
Cartaya,1251.0,31.0,108.0,20083.0,154.359408,537.768262,11.0
Almonaster la Real,63.0,1.0,8.0,1788.0,55.928412,447.427293,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,283.0,1.0,11.0,3761.0,26.588673,292.475405,1.0,0.090909
Almonaster la Real,63.0,1.0,8.0,1788.0,55.928412,447.427293,0.0,0.125000
Villarrasa,147.0,1.0,7.0,2211.0,45.228403,316.598824,0.0,0.142857
Palos de la Frontera,570.0,10.0,36.0,11742.0,85.164367,306.591722,1.0,0.277778
Palma del Condado (La),1028.0,5.0,18.0,10801.0,46.292010,166.651236,0.0,0.277778
Cartaya,1251.0,31.0,108.0,20083.0,154.359408,537.768262,11.0,0.287037
Rociana del Condado,495.0,21.0,61.0,7939.0,264.516942,768.358735,0.0,0.344262
Niebla,192.0,3.0,8.0,4116.0,72.886297,194.363460,0.0,0.375000
Lepe,2069.0,8.0,21.0,27880.0,28.694405,75.322812,5.0,0.380952
